# Step 4

## Quantify WM-task performance with signal detection theory methods

We take in-scanner behavioral data and extract the exact number of specific types of responses to calculate discriminability and bias (d' and c').

In [1]:
import glob
import os

import numpy as np
import pandas as pd
import h5py as h5
import pystan

In [2]:
#f = h5.File('../data/reconstructed_WM.hdf5','r')
f = h5.File('/home/raid2/mfalkiewicz/projects3/dense_connectome/volumes_embedded_full.hdf5')
subjects = f['Working_memory/Subjects']
datapath = '/home/raid2/mfalkiewicz/HCP/data'

## Extract the relevant information from logs

### 2-back

In [34]:
d = np.zeros([len(subjects), 13])
data_2b = pd.DataFrame(d)
data_2b.columns = ['subject_id','hits_RL','misses_RL','fa_RL','cr_RL','hits_LR','misses_LR','fa_LR','cr_LR','hits','misses','fa','cr']
data_2b.subject_id = data_2b.subject_id.astype('str')
data_2b.subject_id = subjects[...]
data_2b.iloc[:,1:] = np.nan

In [35]:
for i, s in enumerate(subjects):
    print "Subject %d of %d" % (i+1, len(subjects))
    try:
        data1 = pd.read_table('%s/%s/unprocessed/3T/tfMRI_WM_RL/LINKED_DATA/EPRIME/%s_3T_WM_run1_TAB.txt' % (datapath, s, s))
        data2 = pd.read_table('%s/%s/unprocessed/3T/tfMRI_WM_LR/LINKED_DATA/EPRIME/%s_3T_WM_run2_TAB.txt' % (datapath, s, s))

        data_2b.hits_RL[i] = sum(np.logical_and(
            np.logical_and(data1['TargetType'] == 'target', data1['Stim.RESP'] == data1['Stim.CRESP']),
                             data1['BlockType'] == '2-Back'))
        data_2b.misses_RL[i] = sum(np.logical_and(
            np.logical_and(data1['TargetType'] == 'target', data1['Stim.RESP'] != data1['Stim.CRESP']),
                               data1['BlockType'] == '2-Back'))
        data_2b.fa_RL[i] = sum(np.logical_and(
                np.logical_and(
                np.logical_or(data1['TargetType'] == 'lure', data1['TargetType'] == 'nontarget'),
                data1['Stim.RESP'] != data1['Stim.CRESP']), data1['BlockType'] == '2-Back'))
        data_2b.cr_RL[i] = 40-data_2b.fa_RL[i]-data_2b.hits_RL[i]-data_2b.misses_RL[i]

        data_2b.hits_LR[i] = sum(np.logical_and(
            np.logical_and(data2['TargetType'] == 'target', data2['Stim.RESP'] == data2['Stim.CRESP']),
                             data2['BlockType'] == '2-Back'))
        data_2b.misses_LR[i] = sum(np.logical_and(
            np.logical_and(data2['TargetType'] == 'target', data2['Stim.RESP'] != data2['Stim.CRESP']),
                               data2['BlockType'] == '2-Back'))
        data_2b.fa_LR[i] = sum(np.logical_and(
                np.logical_and(
                np.logical_or(data2['TargetType'] == 'lure', data2['TargetType'] == 'nontarget'),
                data2['Stim.RESP'] != data2['Stim.CRESP']), data2['BlockType'] == '2-Back'))
        data_2b.cr_LR[i] = 40-data_2b.fa_LR[i]-data_2b.hits_LR[i]-data_2b.misses_LR[i]
    except:
        print "Data for subject %s does not exist" % s

data_2b.hits = data_2b.hits_RL + data_2b.hits_LR
data_2b.misses = data_2b.misses_RL + data_2b.misses_LR
data_2b.fa = data_2b.fa_RL + data_2b.fa_LR
data_2b.cr = data_2b.cr_RL + data_2b.cr_LR

Subject 1 of 494


/home/raid2/mfalkiewicz/cosyenv/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/raid2/mfalkiewicz/cosyenv/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/raid2/mfalkiewicz/cosyenv/lib/python2.7/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/raid2/mfalkiewicz/cosyenv/lib/python2.7/site-packages/ipykernel/__main__.py:17: SettingWithCopy

Subject 2 of 494
Subject 3 of 494
Subject 4 of 494
Subject 5 of 494
Subject 6 of 494
Subject 7 of 494
Subject 8 of 494
Subject 9 of 494
Subject 10 of 494
Subject 11 of 494
Subject 12 of 494
Subject 13 of 494
Data for subject 103515 does not exist
Subject 14 of 494
Subject 15 of 494
Subject 16 of 494
Subject 17 of 494
Subject 18 of 494
Subject 19 of 494
Subject 20 of 494
Subject 21 of 494
Subject 22 of 494
Subject 23 of 494
Subject 24 of 494
Subject 25 of 494
Subject 26 of 494
Subject 27 of 494
Subject 28 of 494
Subject 29 of 494
Subject 30 of 494
Subject 31 of 494
Subject 32 of 494
Subject 33 of 494
Subject 34 of 494
Subject 35 of 494
Subject 36 of 494
Subject 37 of 494
Subject 38 of 494
Subject 39 of 494
Subject 40 of 494
Subject 41 of 494
Subject 42 of 494
Subject 43 of 494
Subject 44 of 494
Subject 45 of 494
Subject 46 of 494
Subject 47 of 494
Subject 48 of 494
Subject 49 of 494
Subject 50 of 494
Subject 51 of 494
Subject 52 of 494
Subject 53 of 494
Data for subject 121618 does not 

In [36]:
data_2b.to_csv('../data/WM_2-back_performance.csv')

### 0-back

In [10]:
d = np.zeros([494, 13])
data_0b = pd.DataFrame(d)
data_0b.columns = ['subject_id','hits_RL','misses_RL','fa_RL','cr_RL','hits_LR','misses_LR','fa_LR','cr_LR','hits','misses','fa','cr']
data_0b.subject_id = data_0b.subject_id.astype('str')
data_0b.subject_id = subjects[...]
data_0b.iloc[:,1:] = np.nan

In [13]:
for i, s in enumerate(subjects):
    print "Subject %d of %d" % (i+1, len(subjects))
    try:
        data1 = pd.read_table('%s/%s/unprocessed/3T/tfMRI_WM_RL/LINKED_DATA/EPRIME/%s_3T_WM_run1_TAB.txt' % (datapath, s, s))
        data2 = pd.read_table('%s/%s/unprocessed/3T/tfMRI_WM_LR/LINKED_DATA/EPRIME/%s_3T_WM_run2_TAB.txt' % (datapath, s, s))

        data_0b.hits_RL[i] = sum(np.logical_and(
            np.logical_and(data1['TargetType'] == 'target', data1['Stim.RESP'] == data1['Stim.CRESP']),
                             data1['BlockType'] == '0-Back'))
        data_0b.misses_RL[i] = sum(np.logical_and(
            np.logical_and(data1['TargetType'] == 'target', data1['Stim.RESP'] != data1['Stim.CRESP']),
                               data1['BlockType'] == '0-Back'))
        data_0b.fa_RL[i] = sum(np.logical_and(
                np.logical_and(
                np.logical_or(data1['TargetType'] == 'lure', data1['TargetType'] == 'nontarget'),
                data1['Stim.RESP'] != data1['Stim.CRESP']), data1['BlockType'] == '0-Back'))
        data_0b.cr_RL[i] = 40-data_0b.fa_RL[i]-data_0b.hits_RL[i]-data_0b.misses_RL[i]

        data_0b.hits_LR[i] = sum(np.logical_and(
            np.logical_and(data2['TargetType'] == 'target', data2['Stim.RESP'] == data2['Stim.CRESP']),
                             data2['BlockType'] == '0-Back'))
        data_0b.misses_LR[i] = sum(np.logical_and(
            np.logical_and(data2['TargetType'] == 'target', data2['Stim.RESP'] != data2['Stim.CRESP']),
                               data2['BlockType'] == '0-Back'))
        data_0b.fa_LR[i] = sum(np.logical_and(
                np.logical_and(
                np.logical_or(data2['TargetType'] == 'lure', data2['TargetType'] == 'nontarget'),
                data2['Stim.RESP'] != data2['Stim.CRESP']), data2['BlockType'] == '0-Back'))
        data_0b.cr_LR[i] = 40-data_0b.fa_LR[i]-data_0b.hits_LR[i]-data_0b.misses_LR[i]
    except:
        print "Data for subject %s does not exist" % s

data_0b.hits = data_0b.hits_RL + data_0b.hits_LR
data_0b.misses = data_0b.misses_RL + data_0b.misses_LR
data_0b.fa = data_0b.fa_RL + data_0b.fa_LR
data_0b.cr = data_0b.cr_RL + data_0b.cr_LR

Subject 1 of 494


/home/raid2/mfalkiewicz/cosyenv/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/raid2/mfalkiewicz/cosyenv/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/raid2/mfalkiewicz/cosyenv/lib/python2.7/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/raid2/mfalkiewicz/cosyenv/lib/python2.7/site-packages/ipykernel/__main__.py:17: SettingWithCopy

Subject 2 of 494
Subject 3 of 494
Subject 4 of 494
Subject 5 of 494
Subject 6 of 494
Subject 7 of 494
Subject 8 of 494
Subject 9 of 494
Subject 10 of 494
Subject 11 of 494
Subject 12 of 494
Subject 13 of 494
Data for subject 103515 does not exist
Subject 14 of 494
Subject 15 of 494
Subject 16 of 494
Subject 17 of 494
Subject 18 of 494
Subject 19 of 494
Subject 20 of 494
Subject 21 of 494
Subject 22 of 494
Subject 23 of 494
Subject 24 of 494
Subject 25 of 494
Subject 26 of 494
Subject 27 of 494
Subject 28 of 494
Subject 29 of 494
Subject 30 of 494
Subject 31 of 494
Subject 32 of 494
Subject 33 of 494
Subject 34 of 494
Subject 35 of 494
Subject 36 of 494
Subject 37 of 494
Subject 38 of 494
Subject 39 of 494
Subject 40 of 494
Subject 41 of 494
Subject 42 of 494
Subject 43 of 494
Subject 44 of 494
Subject 45 of 494
Subject 46 of 494
Subject 47 of 494
Subject 48 of 494
Subject 49 of 494
Subject 50 of 494
Subject 51 of 494
Subject 52 of 494
Subject 53 of 494
Data for subject 121618 does not 

In [14]:
data_0b.to_csv('../data/WM_0-back_performance.csv')

## Use the extracted information to calculate d' and c'

The SDT parameters are estimated for each condition separately. We apply a hierarchical bayesian model described in Lee & Wagenmakers (2014).

Lee, M.D. and Wagenmakers, E.J., 2014. Bayesian cognitive modeling: A practical course. Cambridge University Press.

In [37]:
data_0b = pd.read_csv('../data/WM_0-back_performance.csv', index_col = 0)
data_2b = pd.read_csv('../data/WM_2-back_performance.csv', index_col = 0)

data_0b = data_0b.dropna()
data_2b = data_2b.dropna()

data_0b.iloc[:,1:] = data_0b.iloc[:,1:].astype(int)
data_2b.iloc[:,1:] = data_2b.iloc[:,1:].astype(int)

data_0b = data_0b.reset_index(drop=True)
data_2b = data_2b.reset_index(drop=True)

In [ ]:
SDT_model = '../fgrad/SDT_single.stan'

### Sampling for each run separately

In [ ]:
stan_2b_RL_data = {'h': data_2b['hits_RL'],
                  'f' : data_2b['fa_RL'],
                  's' : data_2b['hits_RL'] + data_2b['misses_RL'],
                  'n' : data_2b['fa_RL'] + data_2b['cr_RL'],
                  'k' : data_2b.shape[0]}

sdt_2b_RL = pystan.stan(file=SDT_model, data=stan_2b_RL_data, iter=10000, chains=4, seed = 123)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_8079f2c55ca4cc5bc0e2862533a84d47 NOW.


In [ ]:
stan_2b_LR_data = {'h': data_2b['hits_LR'],
                  'f' : data_2b['fa_LR'],
                  's' : data_2b['hits_LR'] + data_2b['misses_LR'],
                  'n' : data_2b['fa_LR'] + data_2b['cr_LR'],
                  'k' : data_2b.shape[0]}

sdt_2b_LR = pystan.stan(file=SDT_model, data=stan_2b_LR_data, iter=10000, chains=4, seed = 123)

In [ ]:
stan_0b_RL_data = {'h': data_0b['hits_RL'],
                  'f' : data_0b['fa_RL'],
                  's' : data_0b['hits_RL'] + data_0b['misses_RL'],
                  'n' : data_0b['fa_RL'] + data_0b['cr_RL'],
                  'k' : data_0b.shape[0]}

sdt_0b_RL = pystan.stan(file=SDT_model, data=stan_0b_RL_data, iter=10000, chains=4, seed = 123)

In [ ]:
stan_0b_LR_data = {'h': data_0b['hits_LR'],
                  'f' : data_0b['fa_LR'],
                  's' : data_0b['hits_LR'] + data_0b['misses_LR'],
                  'n' : data_0b['fa_LR'] + data_0b['cr_LR'],
                  'k' : data_0b.shape[0]}

sdt_0b_LR = pystan.stan(file=SDT_model, data=stan_0b_LR_data, iter=10000, chains=4, seed = 123)

### Assemble the means into a single data table

In [ ]:
SDT_nback = pd.DataFrame(data_2b['subject_id'])
SDT_nback['dprime_2b_RL'] = np.nan
SDT_nback['dprime_2b_LR'] = np.nan
SDT_nback['dprime_0b_RL'] = np.nan
SDT_nback['dprime_0b_LR'] = np.nan
SDT_nback['bias_2b_RL'] = np.nan
SDT_nback['bias_2b_LR'] = np.nan
SDT_nback['bias_0b_RL'] = np.nan
SDT_nback['bias_0b_LR'] = np.nan

In [ ]:
sdt_2b_RL_params = sdt_2b_RL.extract(permuted = False, inc_warmup=False)
sdt_2b_RL_means = np.mean(np.mean(sdt_2b_RL_params[:,:,4:-1],axis = 0), axis = 0)
SDT_nback['dprime_2b_RL'] = sdt_2b_RL_means[460:]
SDT_nback['bias_2b_RL'] = sdt_2b_RL_means[0:460]

In [ ]:
sdt_2b_LR_params = sdt_2b_LR.extract(permuted = False, inc_warmup=False)
sdt_2b_LR_means = np.mean(np.mean(sdt_2b_LR_params[:,:,4:-1],axis = 0), axis = 0)
SDT_nback['dprime_2b_LR'] = sdt_2b_LR_means[460:]
SDT_nback['bias_2b_LR'] = sdt_2b_LR_means[0:460]

In [ ]:
sdt_0b_RL_params = sdt_0b_RL.extract(permuted = False, inc_warmup=False)
sdt_0b_RL_means = np.mean(np.mean(sdt_0b_RL_params[:,:,4:-1],axis = 0), axis = 0)
SDT_nback['dprime_0b_RL'] = sdt_0b_RL_means[460:]
SDT_nback['bias_0b_RL'] = sdt_0b_RL_means[0:460]

In [ ]:
sdt_0b_LR_params = sdt_0b_LR.extract(permuted = False, inc_warmup=False)
sdt_0b_LR_means = np.mean(np.mean(sdt_0b_LR_params[:,:,4:-1],axis = 0), axis = 0)
SDT_nback['dprime_0b_LR'] = sdt_0b_LR_means[460:]
SDT_nback['bias_0b_LR'] = sdt_0b_LR_means[0:460]

In [ ]:
SDT_nback.to_csv('../data/WM_SDT.csv')